### Installing the CPOR repo

In [1]:
!pip install --upgrade --force-reinstall git+https://github.com/aiplan4eu/up-cpor

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/aiplan4eu/up-cpor to /tmp/pip-req-build-9yvl1osy
  Running command git clone --filter=blob:none --quiet https://github.com/aiplan4eu/up-cpor /tmp/pip-req-build-9yvl1osy
  Resolved https://github.com/aiplan4eu/up-cpor to commit da586943aa3d3ee71956326f02117f192a58238a
  Preparing metadata (setup.py) ... done
  Using cached pythonnet-3.0.0-py3-none-any.whl (278 kB)
  Using cached clr_loader-0.2.5-py3-none-any.whl (51 kB)
  Using cached cffi-1.15.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (441 kB)
  Using cached pycparser-2.21-py2.py3-none-any.whl (118 kB)
Reason for being yanked: Is missing python_requires and thus installs on unsupported versions
  Created wheel for up-cpor: filename=up_cpor-1.0.0-py3-none-any.whl size=195302 sha256=f94f96481a54a7d44d7471aae813a9de9575fb038ea239b9de15e6d84e3ad140
  Stored in directory: /tmp/pip-ephem-wheel-cac

In [2]:
!pip show up-cpor
# !ls -R /usr/local/lib/python3.9/dist-packages | grep "cpor"
!ls -R /usr/local/lib/python3.9/dist-packages | grep "CPOR"

Name: up-cpor
Version: 1.0.0
Summary: up_cpor
Home-page: 
Author: BGU CPOR Development Team
Author-email: shanigu@bgu.ac.il
License: APACHE
Location: /usr/local/lib/python3.10/dist-packages
Requires: pythonnet
Required-by: 
ls: cannot access '/usr/local/lib/python3.9/dist-packages': No such file or directory


### Installing the UP repo

In [3]:
!pip install git+https://github.com/aiplan4eu/unified-planning@action-selector-mixin


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/aiplan4eu/unified-planning (to revision action-selector-mixin) to /tmp/pip-req-build-pqm_4suy
  Running command git clone --filter=blob:none --quiet https://github.com/aiplan4eu/unified-planning /tmp/pip-req-build-pqm_4suy
  Running command git checkout -b action-selector-mixin --track origin/action-selector-mixin
  Switched to a new branch 'action-selector-mixin'
  Branch 'action-selector-mixin' set up to track remote branch 'action-selector-mixin' from 'origin'.
  Resolved https://github.com/aiplan4eu/unified-planning to commit 0a6fe4420117e6b46f9e50162e846e5a34bb301c
  Preparing metadata (setup.py) ... done


### Installing the planner

In [4]:
!pysmt-install --z3

This script allows you to install the solvers supported by pySMT.

By executing this script, you confirm that you have read and agreed
with the licenses of each solver.

Notice: the installation process might require building tools
        (e.g., make and gcc).

Continue? [Y]es/[N]o: n


### Make sure that colab can use mono

In [5]:
!sudo apt-get install -y mono-complete

Reading package lists... Done
Building dependency tree       
Reading state information... Done
mono-complete is already the newest version (6.8.0.105+dfsg-2).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.


In [6]:
!mono --version

Mono JIT compiler version 6.8.0.105 (Debian 6.8.0.105+dfsg-2 Wed Feb 26 23:23:50 UTC 2020)
Copyright (C) 2002-2014 Novell, Inc, Xamarin Inc and Contributors. www.mono-project.com
	TLS:           __thread
	SIGSEGV:       altstack
	Notifications: epoll
	Architecture:  amd64
	Disabled:      none
	Misc:          softdebug 
	Interpreter:   yes
	LLVM:          supported, not enabled.
	Suspend:       hybrid
	GC:            sgen (concurrent by default)


# Load Problems

### The problems PDDL files can be found in https://github.com/aiplan4eu/up-cpor/tree/master/Tests

In [7]:
from unified_planning.io import PDDLReader

reader = PDDLReader()

# Parsing a PDDL problem from file
problem = reader.parse_problem(
    f"d.pddl",
    f"p.pddl"
)

# CPOR Engine Example Run

In [19]:
import unified_planning.environment as environment
from unified_planning.shortcuts import OneshotPlanner
from unified_planning.engines.results import PlanGenerationResultStatus

env = environment.get_environment()
env.factory.add_engine('CPORPlanning', 'up_cpor.engine', 'CPORImpl')

with OneshotPlanner(name='CPORPlanning') as planner:
    result = planner.solve(problem)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{planner.name} found a valid plan!')
        print(f'Success')
    else:
        print('No plan found!')

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 9 of `<ipython-input-19-0ee5bfb2cc1b>`, you are using the following planning engine:
  * Engine name: CPOR
  * Developers:  Guy Shani
  * Description: CPOR is an offline contingent planner.
  *               It computes a complete plan tree (or graph) where each node is labeled by an action, and edges are labeled by observations.
  *              The leaves of the plan tree correspond to goal states.

CPORPlanning found a valid plan!
Success


# CPOR Meta-Engine Example Run With 'up-tamer' as Inner Planner

### Must install inner solver

In [12]:
!pip install unified-planning[tamer]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import unified_planning.environment as environment
from unified_planning.engines.results import PlanGenerationResultStatus
from unified_planning.shortcuts import OneshotPlanner

env = environment.get_environment()
env.factory.add_meta_engine('MetaCPORPlanning', 'up_cpor.engine', 'CPORMetaEngineImpl')

with OneshotPlanner(name='MetaCPORPlanning[tamer]') as planner:
    result = planner.solve(problem)
    if result.status == PlanGenerationResultStatus.SOLVED_SATISFICING:
        print(f'{planner.name} found a valid plan!')
        print(f'Success')
    else:
        print('No plan found!')

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 9 of `<ipython-input-20-f69e3db481b2>`, you are using the following planning engine:
  * Engine name: Conitngent Planning Algorithms
  * Developers:  Guy Shani
  * Description: Algorithms for offline and online decision making under partial observability and sensing actions

CPORPlanning[Tamer] found a valid plan!
Success


# SDR Engine Example Run with UP Simulated Env

In [16]:
import unified_planning.environment as environment
from unified_planning.model.contingent.environment import SimulatedEnvironment
from unified_planning.shortcuts import ActionSelector

env = environment.get_environment()
env.factory.add_engine('SDRPlanning', 'up_cpor.engine', 'SDRImpl')

with ActionSelector(name='SDRPlanning', problem=problem) as solver:
    simulatedEnv = SimulatedEnvironment(problem)
    while not simulatedEnv.is_goal_reached():
        action = solver.get_action()
        observation = simulatedEnv.apply(action)
        solver.update(observation)

  *** Credits ***
  * In operation mode `ActionSelector` at line 9 of `<ipython-input-16-c8c4086bc06c>`, you are using the following planning engine:
  * Engine name: SDR
  * Developers:  Guy Shani
  * Description: SDR is an online contingent replanner.
  *              It provides one action at a time, and then awaits to receive an observation from the environment.



/usr/local/lib/python3.10/dist-packages/unified_planning/engines/mixins/sequential_simulator.py:50: UserWarning: We cannot establish whether sequential_simulator is able to handle this problem!
  warn(msg)
/usr/local/lib/python3.10/dist-packages/unified_planning/engines/sequential_simulator.py:67: UserWarning: The Grounder used in the <class 'unified_planning.engines.sequential_simulator.UPSequentialSimulator'> does not support the given problem
  warn(msg)


move p1-1 p1-2
move p1-2 p1-3
move p1-3 p1-4
move p1-4 p2-4
smell_wumpus p2-4
move p2-4 p2-5
move p2-5 p3-5
feel-breeze p3-5
smell_wumpus p3-5
move p3-5 p3-4
move p3-4 p4-4
move p4-4 p5-4
move p5-4 p5-5
grab p5-5


# SDR Engine Example Run with SDR Simulated Env

In [17]:
import unified_planning.environment as environment
from unified_planning.shortcuts import ActionSelector

from up_cpor.simulator import SDRSimulator


env = environment.get_environment()
env.factory.add_engine('SDRPlanning', 'up_cpor.engine', 'SDRImpl')

with ActionSelector(name='SDRPlanning', problem=problem) as solver:
    simulatedEnv = SDRSimulator(problem)
    while not simulatedEnv.is_goal_reached():
        action = solver.get_action()
        observation = simulatedEnv.apply(action)
        solver.update(observation)

  *** Credits ***
  * In operation mode `ActionSelector` at line 11 of `<ipython-input-17-daa1145251f1>`, you are using the following planning engine:
  * Engine name: SDR
  * Developers:  Guy Shani
  * Description: SDR is an online contingent replanner.
  *              It provides one action at a time, and then awaits to receive an observation from the environment.

move p1-1 p1-2
move p1-2 p1-3
feel-breeze p1-3
smell_wumpus p1-3
move p1-3 p2-3
move p2-3 p3-3
smell_wumpus p3-3
move p3-3 p2-3
move p2-3 p2-4
feel-breeze p2-4
move p2-4 p2-3
move p2-3 p3-3
move p3-3 p4-3
move p4-3 p4-2
smell_wumpus p4-2
move p4-2 p5-2
move p5-2 p5-3
feel-breeze p5-3
smell_wumpus p5-3
move p5-3 p5-4
move p5-4 p5-5
grab p5-5
